# Comparative Analysis of Neural Network Optimization Methods

## A Study of SGD, Adam, and L-BFGS Convergence Behavior on the Friedman Dataset

**Overview:**
This notebook explores the performance characteristics of different optimization algorithms (SGD, Adam, L-BFGS) 

**Key Objectives:**
- Compare convergence rates of different optimizers
- Analyze the impact of different parameters

In [ ]:
# Import required libraries
import numpy as np
import torch

# Set random seeds for reproducibility
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Configure hardware settings (CPU/GPU)
device = torch.device("cpu")
print(f"Using device: {device}")

# Dataset Configuration

The Friedman dataset is a synthetic dataset commonly used for regression problems and benchmarking machine learning algorithms.

**Dataset Characteristics:**
- **Type**: Synthetic regression dataset
- **Features**: {N_FEATURES} input features, but only 5 are actually used in the target function
- **Target Function**: y = 10 * sin(πx₁x₂) + 20(x₃ - 0.5)² + 10x₄ + 5x₅ + ε
- **Noise (ε)**: Gaussian noise with standard deviation {NOISE_LEVEL}


In [ ]:
# Dataset Configuration

# Define parameters for the Friedman dataset
N_SAMPLES = 10000  # Total number of data points to generate
N_FEATURES = 16  # Dimensionality of input features
NOISE_LEVEL = 0.001  # Amount of noise added to make the problem more realistic

# Generate the Friedman dataset
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split

X, y = make_friedman1(n_samples=N_SAMPLES, n_features=N_FEATURES, noise=NOISE_LEVEL, random_state=RANDOM_SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# # Generate linear regression dataset

# X = np.random.randn(N_SAMPLES, N_FEATURES)
# w = np.random.randn(N_FEATURES)
# b = np.random.randn()
# y = X @ w + b + np.random.randn(N_SAMPLES) * NOISE_LEVEL

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=RANDOM_SEED
# )

# Convert numpy arrays to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).reshape(-1, 1)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test).reshape(-1, 1)

# Print dataset shapes
print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}")

# Model Architecture Implementation
Implement SimpleMLP class - a basic 2-layer neural network with configurable hidden layer size.

In [ ]:
# Model Architecture Implementation

import torch.nn as nn

class SimpleMLP(nn.Module):
    def __init__(self, input_size=N_FEATURES, hidden_size=64, seed=RANDOM_SEED):
        super(SimpleMLP, self).__init__()
        torch.manual_seed(seed)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        # self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model to verify
model = SimpleMLP().to(device)
print(model)

# Data Generation and Preprocessing
Generate Friedman dataset, split into train/test sets, and prepare PyTorch DataLoaders for efficient batch processing.

In [ ]:
# Data Preparation for PyTorch

from torch.utils.data import DataLoader, TensorDataset

# Create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print DataLoader information
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

# Training Framework Implementation
Define model evaluation function and main training loop with support for different optimizers (SGD, Adam, LBFGS).

In [ ]:
# Training Framework Implementation

import torch.optim as optim
import time
from tqdm import tqdm
import gc

# Define model evaluation function
def evaluate_model(model, data_loader):
    model.eval()
    running_loss = 0.0
    criterion = nn.MSELoss()

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()

    return running_loss / len(data_loader)

# Main training function
def train_network(optimizer_name, model_class=SimpleMLP, hidden_size=64, max_time=300):
    gc.collect()

    model = model_class(hidden_size=hidden_size).to(device)
    criterion = nn.MSELoss()
    eval_count = 0

    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=0.01)
    elif optimizer_name == "LBFGS":
        optimizer = optim.LBFGS(model.parameters(), lr=0.05)

    num_epochs = 10000
    train_losses = []
    test_losses = []
    timestamps = []
    start_time = time.time()

    for epoch in tqdm(range(num_epochs), desc=f"Training with {optimizer_name}"):
        current_time = time.time() - start_time
        if current_time > max_time:
            print(f"Stopping training: exceeded time limit of {max_time} seconds")
            break

        model.train()
        running_loss = 0.0
        test_loss = evaluate_model(model, test_loader)

        if optimizer_name == "LBFGS":
            def closure():
                optimizer.zero_grad()
                total_loss = 0
                nonlocal running_loss, eval_count
                eval_count += 1

                for data, target in train_loader:
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    loss = criterion(output, target)
                    loss.backward()
                    total_loss += loss.item()
                    running_loss += loss.item()
                running_loss = running_loss / len(train_loader)
                return total_loss

            optimizer.step(closure)
        else:
            for data, target in train_loader:
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                eval_count += 1

            running_loss = running_loss / len(train_loader)

        train_loss = running_loss
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        timestamps.append(current_time)

        if optimizer_name == "LBFGS" and epoch % 5 == 0:
            print(f"{optimizer_name} Epoch {epoch}: Train MSE={train_loss:.6f}, Test MSE={test_loss:.6f}, Time={current_time:.2f}s")

    training_time = time.time() - start_time
    return train_losses, test_losses, training_time, eval_count, timestamps

# Optimizer Comparison Experiment
Implement parameter sweep experiment to compare optimizers across different network architectures.

In [ ]:
# Optimizer Comparison Experiment

import matplotlib.pyplot as plt
from collections import defaultdict

# Function to plot convergence comparison
def plot_convergence_comparison(optimizer_results, hidden_size):
    plt.figure(figsize=(10, 6))

    for opt_name, (train_losses, test_losses, _, _, timestamps) in optimizer_results.items():
        plt.plot(timestamps, train_losses, "-", label=f"{opt_name} (train)")

    plt.title(f"Training Loss Convergence - Hidden Size {hidden_size}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("MSE Loss")
    plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.show()

# Main experiment function
def parameter_sweep_experiment(hidden_sizes=[16, 64, 256, 1024], max_time=20):
    results = defaultdict(list)

    for hidden_size in hidden_sizes:
        print(f"\nExperimenting with hidden_size={hidden_size}")
        current_results = {}

        for opt in ["LBFGS", "SGD", "Adam"]:
            try:
                print(f"\nTraining {opt} with hidden_size={hidden_size}")
                train_losses, test_losses, training_time, eval_count, timestamps = train_network(opt, hidden_size=hidden_size, max_time=max_time)

                current_results[opt] = (train_losses, test_losses, training_time, eval_count, timestamps)

                results[opt].append({
                    "hidden_size": hidden_size,
                    "final_train_mse": train_losses[-1],
                    "final_test_mse": test_losses[-1],
                    "training_time": training_time,
                    "eval_count": eval_count,
                })
            except RuntimeError as e:
                print(f"Error training with {opt}: {e}")
                continue

        plot_convergence_comparison(current_results, hidden_size)

    return results

# Function to plot parameter sweep results
def plot_parameter_sweep_results(results):
    plt.figure(figsize=(8, 5))

    for opt in results:
        hidden_sizes = [r["hidden_size"] for r in results[opt]]
        mse = [r["final_test_mse"] for r in results[opt]]
        plt.plot(hidden_sizes, mse, "o-", label=opt)
    plt.xlabel("Hidden Layer Size")
    plt.ylabel("Final Test MSE")
    plt.title("Model Size vs MSE")
    plt.legend()

    plt.tight_layout()
    plt.show()

# Run the parameter sweep experiment
print("\nStarting parameter sweep experiment...")
sweep_results = parameter_sweep_experiment()
plot_parameter_sweep_results(sweep_results)

### Task 1: Explore the Experimental Setup
1. Review the provided code and understand the dataset, model architecture, and training setup.
2. Run the code and observe the results.
3. Identify key parameters that might influence the performance of SGD, Adam, and L-BFGS.  
4. Document your hypotheses in a table, make a hypothesis of how these parameters affect the algorithms' performance.

---

### Task 2: Test Your Hypotheses
1. Modify one parameter at a time (e.g., dataset size, noise level, hidden size) and observe how the results change.
2. Compare the outcomes with your hypotheses. Were your predictions correct? Explain why or why not.
3. Reflect on which optimization problems or setups might suit each algorithm better and why.

---

### Task 3: Linear Regression Analysis
1. Uncomment the code for generating the linear regression dataset.
2. Repeat the steps from Task 1 and Task 2 using this simpler dataset.
3. Compare how the optimizers behave on linear regression versus the Friedman dataset and note any differences.
